## For 3 categories

In [1]:
import pandas as pd
import numpy as np

In [2]:
data=pd.read_excel("sample tweets.xlsx")

In [3]:
X=data['SentimentText']
Y=data['Sentiment']

In [6]:
with open('../glove.txt',encoding='mbcs') as f:
    doc=f.readlines()
    
glove_emb={
          }
for i in range(len(doc)):
    word   = doc[i].split(" ")[0]
    vector = np.array(doc[i].split(" ")[1:],dtype='float32')
    
    glove_emb[word]=vector

In [7]:
glove_emb['the']

array([ 4.1800e-01,  2.4968e-01, -4.1242e-01,  1.2170e-01,  3.4527e-01,
       -4.4457e-02, -4.9688e-01, -1.7862e-01, -6.6023e-04, -6.5660e-01,
        2.7843e-01, -1.4767e-01, -5.5677e-01,  1.4658e-01, -9.5095e-03,
        1.1658e-02,  1.0204e-01, -1.2792e-01, -8.4430e-01, -1.2181e-01,
       -1.6801e-02, -3.3279e-01, -1.5520e-01, -2.3131e-01, -1.9181e-01,
       -1.8823e+00, -7.6746e-01,  9.9051e-02, -4.2125e-01, -1.9526e-01,
        4.0071e+00, -1.8594e-01, -5.2287e-01, -3.1681e-01,  5.9213e-04,
        7.4449e-03,  1.7778e-01, -1.5897e-01,  1.2041e-02, -5.4223e-02,
       -2.9871e-01, -1.5749e-01, -3.4758e-01, -4.5637e-02, -4.4251e-01,
        1.8785e-01,  2.7849e-03, -1.8411e-01, -1.1514e-01, -7.8581e-01],
      dtype=float32)

In [8]:
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
rg=RegexpTokenizer("[a-zA-Z]+")
sw=stopwords.words("english")

In [9]:
unique={
    
}
weight=[]
k=1;
X_train=np.zeros((10000,45))
X_train=X_train.astype('int32')
for i in range(X.shape[0]):
    words=rg.tokenize(X[i])
    words=[w for w in words if w not in sw and w in glove_emb and len(w)>2]
    
    for j in range(min(len(words),45)):
        if words[j].lower() in unique:
            X_train[i][j]=unique[words[j].lower()]
            
        else:
            weight.append(glove_emb[words[j]])
            X_train[i][j]=k
            unique[words[j].lower()]=k
            k+=1

In [10]:
weight_matrix=[np.zeros((50,))]+weight
weight_matrix=np.array(weight_matrix)

In [11]:
weight_matrix.shape

(4185, 50)

In [12]:
from sklearn.preprocessing import LabelEncoder

In [13]:
le=LabelEncoder()

In [14]:
Y_encoded=le.fit_transform(Y)

In [15]:
from keras.utils import to_categorical
Y_train=to_categorical(Y_encoded)

Using TensorFlow backend.


In [16]:
Y_train

array([[1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.]], dtype=float32)

In [17]:
from keras.models import Sequential
from keras.layers import *

In [18]:
model=Sequential()
model.add(Embedding(4185,50, weights=[weight_matrix], input_length=45, trainable=False))
model.add(LSTM(32,return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(32))
model.add(Dropout(0.2))
model.add(Dense(3,activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 45, 50)            209250    
_________________________________________________________________
lstm_1 (LSTM)                (None, 45, 32)            10624     
_________________________________________________________________
dropout_1 (Dropout)          (None, 45, 32)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 99        
Total params: 228,293
Trainable params: 19,043
Non-trainable params: 209,250
___________________________________________

In [19]:
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=['acc'])

In [ ]:
hist=model.fit(X_train,Y_train,validation_split=.2,epochs=15,batch_size=32)

Train on 8000 samples, validate on 2000 samples
Epoch 1/15
8000/8000 [==============================] - 15s 2ms/step - loss: 0.6980 - acc: 0.7024 - val_loss: 0.7080 - val_acc: 0.7060
Epoch 2/15
8000/8000 [==============================] - 15s 2ms/step - loss: 0.5231 - acc: 0.7951 - val_loss: 0.7038 - val_acc: 0.7320
Epoch 3/15
8000/8000 [==============================] - 15s 2ms/step - loss: 0.4603 - acc: 0.8295 - val_loss: 0.6375 - val_acc: 0.7470
Epoch 4/15
8000/8000 [==============================] - 15s 2ms/step - loss: 0.4235 - acc: 0.8436 - val_loss: 0.6248 - val_acc: 0.7575
Epoch 5/15
8000/8000 [==============================] - 15s 2ms/step - loss: 0.4107 - acc: 0.8478 - val_loss: 0.6230 - val_acc: 0.7515
Epoch 6/15
8000/8000 [==============================] - 14s 2ms/step - loss: 0.3946 - acc: 0.8484 - val_loss: 0.6594 - val_acc: 0.7505s: 0.3966 - acc: 0.
Epoch 7/15
8000/8000 [==============================] - 15s 2ms/step - loss: 0.3713 - acc: 0.8624 - val_loss: 0.6081 - val_

In [ ]:
X_train.shape ,Y_train.shape